In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
base_path = "/Users/dzz1th/Job/mgi/Soroka/data/pc_data/"
employment_levels_path = base_path + "employment_level_pairwise_scores.csv"
employment_dynamics_path = base_path + "employment_dynamics_pairwise_scores.csv"
inflation_levels_path = base_path + "inflation_level_pairwise_scores.csv"
inflation_dynamics_path = base_path + "inflation_dynamics_pairwise_scores.csv"
interest_rate_path = base_path + "interest_rate_trajectory_pairwise_scores.csv"
balance_sheet_path = base_path + "balance_sheet_trajectory_pairwise_scores.csv"
guidance_path = base_path + "forward_guidance_guidance_pairwise_scores.csv"


employment_levels_df = pd.read_csv(employment_levels_path)
employment_dynamics_df = pd.read_csv(employment_dynamics_path)
inflation_levels_df = pd.read_csv(inflation_levels_path)
inflation_dynamics_df = pd.read_csv(inflation_dynamics_path)
interest_rate_df = pd.read_csv(interest_rate_path)
balance_sheet_df = pd.read_csv(balance_sheet_path)
guidance_df = pd.read_csv(guidance_path)

In [5]:
from functools import reduce

employment_levels_df = employment_levels_df[['date', 'employment_level_score']]
employment_dynamics_df = employment_dynamics_df[['date', 'employment_dynamics_score']]
inflation_levels_df = inflation_levels_df[['date', 'inflation_level_score']]
inflation_dynamics_df = inflation_dynamics_df[['date', 'inflation_dynamics_score']]
interest_rate_df = interest_rate_df[['date', 'interest_rate_trajectory_score']]
balance_sheet_df = balance_sheet_df[['date', 'balance_sheet_trajectory_score']]
guidance_df = guidance_df[['date', 'forward_guidance_guidance_score']]

# Create a single dataframe with all score columns
df = pd.DataFrame({
    'date': employment_levels_df['date'],
    'employment_level_score': employment_levels_df['employment_level_score'],
    'employment_dynamics_score': employment_dynamics_df['employment_dynamics_score'],
    'inflation_level_score': inflation_levels_df['inflation_level_score'],
    'inflation_dynamics_score': inflation_dynamics_df['inflation_dynamics_score'],
    'interest_rate_trajectory_score': interest_rate_df['interest_rate_trajectory_score'],
    'balance_sheet_trajectory_score': balance_sheet_df['balance_sheet_trajectory_score'],
    'forward_guidance_guidance_score': guidance_df['forward_guidance_guidance_score']
})


df['date'] = pd.to_datetime(df['date'])





AttributeError: 'NoneType' object has no attribute 'encode'

In [8]:
### Some magic regarding fixing the inflation scores

## Invert Employment Level Scores
df['employment_level_score'] = -1 * df['employment_level_score']

## Invert Employment Dynamics Scores
df['employment_dynamics_score'] = -1 * df['employment_dynamics_score']

df['inflation_level_score'] = -1 * df['inflation_level_score']
df['inflation_dynamics_score'] = -1 * df['inflation_dynamics_score']


## Invert Balance Sheet Scores completely
df['balance_sheet_trajectory_score'] = -1 * df['balance_sheet_trajectory_score']

In [9]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Convert date to datetime if it's not already
if not pd.api.types.is_datetime64_any_dtype(df['date']):
    df['date'] = pd.to_datetime(df['date'])

# Filter data for our time range (2019-01 to 2024-01)
mask = (df['date'] >= '2019-01-01') & (df['date'] <= '2025-01-31')
filtered_df = df.loc[mask].copy()

# Make sure the data is sorted by date
filtered_df = filtered_df.sort_values('date')

# Key events with dates and descriptions for inflation
key_events = [
    {'date': '2020-03-15', 'description': 'COVID Impact', 'color': 'red'},
    {'date': '2020-09-16', 'description': 'Transitory Inflation', 'color': 'orange'},
    {'date': '2022-07-27', 'description': 'Inflation Peak', 'color': 'purple'},
    {'date': '2024-01-31', 'description': 'Inflation Stabilization', 'color': 'green'}
]

# Create the figure
fig = go.Figure()

# Add inflation level score
fig.add_trace(
    go.Scatter(
        x=filtered_df['date'], 
        y=filtered_df['inflation_level_score'],
        mode='lines+markers',
        name='Inflation Level',
        line=dict(color='#1F77B4', width=2.5),  # Blue for inflation level
        marker=dict(
            size=8,
            color='#1F77B4',
            line=dict(color='white', width=1)
        )
    )
)

# Add reference line at zero
fig.add_shape(
    type="line",
    x0=filtered_df['date'].min(),
    y0=0,
    x1=filtered_df['date'].max(),
    y1=0,
    line=dict(color="black", width=1, dash="dash"),
)

# Add key events as vertical lines with clearly visible annotations
for event in key_events:
    event_date = pd.Timestamp(event['date'])
    
    # Find the nearest date in our data
    nearest_idx = abs(filtered_df['date'] - event_date).argmin()
    
    # Find the inflation score on that date (if available)
    y_position = 0
    if nearest_idx < len(filtered_df):
        y_position = filtered_df.iloc[nearest_idx]['inflation_level_score']
    
    # Add vertical line
    fig.add_shape(
        type="line",
        x0=event_date,
        y0=filtered_df['inflation_level_score'].min() - 0.1,
        x1=event_date,
        y1=filtered_df['inflation_level_score'].max() + 0.1,
        line=dict(
            color=event['color'],
            width=1.5,
            dash="solid",
        ),
    )
    
    # Position adjustments for annotations to prevent overlap and move out of middle
    # Vary the positions based on event date
    y_offsets = {
        '2020-03-15': 0.3,
        '2020-09-16': -0.3,
        '2022-07-27': 0.1,
        '2024-01-31': -0.1
    }
    
    y_offset = y_offsets.get(event['date'], 0)
    
    # Add clear text labels
    fig.add_annotation(
        x=event_date,
        y=y_position + y_offset,
        text=event['description'],
        showarrow=False,
        font=dict(size=12, color=event['color'], family="Arial", weight="bold"),
        align="center",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor=event['color'],
        borderwidth=2,
        borderpad=3,
    )

# Add recession shading for COVID period
fig.add_shape(
    type="rect",
    x0="2020-02-15", 
    x1="2020-07-15",
    y0=filtered_df['inflation_level_score'].min() - 0.2,
    y1=filtered_df['inflation_level_score'].max() + 0.2,
    fillcolor="rgba(128, 128, 128, 0.2)", 
    line=dict(width=0),
    layer="below",
)

# Add clear text label for COVID recession
fig.add_annotation(
    x=pd.Timestamp("2020-04-15"),
    y=filtered_df['inflation_level_score'].max() + 0.2,
    text="COVID Recession",
    showarrow=False,
    font=dict(size=14, color="black"),
    align="center",
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
    borderwidth=1,
    borderpad=3,
)

# Add annotations for inflation level score - moved to edges
fig.add_annotation(
    x=pd.Timestamp("2019-03-01"),
    y=filtered_df['inflation_level_score'].max() * 0.8,
    text="↑ Rising Inflation",
    showarrow=False,
    font=dict(size=14, color="#1F77B4"),
    align="left",
    bgcolor="white",
    bordercolor="#1F77B4",
    borderwidth=1,
    borderpad=3,
)

fig.add_annotation(
    x=pd.Timestamp("2019-03-01"),
    y=filtered_df['inflation_level_score'].min() * 0.8,
    text="↓ Falling Inflation",
    showarrow=False,
    font=dict(size=14, color="#1F77B4"),
    align="left",
    bgcolor="white",
    bordercolor="#1F77B4",
    borderwidth=1,
    borderpad=3,
)

# Update layout with clear, print-friendly styling
fig.update_layout(
    title={
        'text': 'Federal Reserve Inflation Assessment (2019-2024)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20, family="Arial", color="black")
    },
    showlegend=False,  # No legend needed
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=1000,
    margin=dict(l=80, r=60, t=100, b=80)
)

# Update x-axis
fig.update_xaxes(
    title_text="FOMC Meeting Date",
    title_font=dict(size=16, family="Arial", color="black"),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=1,
    dtick="M3",
    tickformat="%b\n%Y",
    tickfont=dict(size=12, family="Arial", color="black"),
    zeroline=False
)

# Update y-axis
fig.update_yaxes(
    title_text="Inflation Level Score",
    title_font=dict(size=16, family="Arial", color="#1F77B4"),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=1,
    zeroline=True,
    zerolinecolor='black',
    zerolinewidth=1,
    tickfont=dict(size=12, family="Arial", color="#1F77B4"),
)

# Show the figure
fig.show()

# Save as a static image for presentations
fig.write_image("fed_inflation_level_assessment.png", scale=2)  # Higher scale for better resolution

# Save as HTML
fig.write_html("fed_inflation_level_assessment.html")